In [ ]:
# sag üst 3 noktada accelerator GPU T4 x2
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics==8.0.196

In [ ]:
#gerekli kutuphanelerin yuklenmesi
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

import cv2
import json
import glob

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
#ROBOFLOW UZERİNDEN ALINAN KOD BLOK BASLANGIC
#dataset-link.txt dosyasını okuyun.
#BITIS

In [ ]:
#TRAIN GPU T4 x2 FOR KAGGLE device=0,1
# yolov8l.pt , yolov8x.pt denenebilir. epoch = 100 denenebilir.
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=800 device=0,1 plots=True 

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
#MODEL DOGRULAMA
%cd {HOME}
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
#MODEL TAHMİN
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True
#conf:guven esigi

In [ ]:
#Birkac ornek gormek icin
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:30]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
!zip -r file.zip /kaggle/working/runs

In [ ]:
#DETECT ISLEMINI YAPIP EKRANA GORSEL GOSTEREN VE DETECT EDİLEN NESNELERIN BILGISINI YAZDIRAN FONKSIYON
def detect(image_path, model_path='/kaggle/working/runs/detect/train/weights/best.pt'):
    # Modeli yükle
    model = YOLO(model_path)
    # Görüntüyü yükle
    image = cv2.imread(image_path)
    imagename_withoutextension = image_path.split('/')[-1]

    # Tespitleri gerçekleştir
    results = model.predict(image_path, imgsz=640, conf=0.25, iou=0.45)
    # Her bir nesne için bilgileri yazdır
    print("-------------------------")
    json_object = results[0].tojson()
    json_array = json.loads(json_object)
    for item in json_array:
        print("Class adı: " + item["name"] + " Class No: " + str(item["class"]) + " Dogruluk: " + str(item["confidence"]))
    print("-------------------------")
    annotated_frame = results[0].plot()
    # Görüntüyü bir dosya olarak kaydet
    cv2.imwrite('annotated_image.jpg', annotated_frame)
    # Kaydedilen dosyayı göster
    display(Image(filename='annotated_image.jpg'))

        

In [ ]:
detect("dosya-yolu")